In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

import sys
sys.path.append("../../../")

from chiseling.dgps.benchtm import BenchTM
from chiseling.source.learners.baselearners_causal import make_causal_random_forest_learner
from chiseling.source.protocol.IRST import UnitRegistrar, IRST
from chiseling.source.protocol.utils import aipw_intercept_pseudo_outcome
from chiseling.source.strategies.data_splitting_strategy import DataSplittingStrategy
from chiseling.source.strategies.alpha_spending_strategy import AlphaSpendingStrategy

from chiseling.benchmark.benchmark import Benchmark

In [3]:
# Hyperparameters
n = 500

alpha = 0.05

random_seed = 0

benchtm_path = "../../../benchtm/"

In [4]:
# Sample data
sampler = BenchTM(scenario=1, benchtm_path=benchtm_path, random_seed=random_seed)
TX, Y = sampler.sample(n)

In [5]:
sampler.get_optimal_region_metrics()

(np.float64(0.2123999999999997),
 np.float64(1.0),
 np.float64(0.2123999999999997),
 0,
 0,
 0)

In [6]:
# Initialize protocol
unit_reg = UnitRegistrar(random_seed)
regTX = unit_reg.register_units(TX)
pY = aipw_intercept_pseudo_outcome(T=TX[:,0], X=TX[:,1:], Y=Y,
                                   propensities=0.5, random_seed=random_seed)
protocol = IRST(regTX, Y, pY=pY, alpha=alpha)

In [7]:
# Make random forest learner
rf_learner = make_causal_random_forest_learner(random_seed=random_seed)

In [8]:
# Run chiseling
strategy = AlphaSpendingStrategy(protocol=protocol,
                                 test_thresh=0,
                                 learner=rf_learner,
                                 n_burn_in=0.2,
                                 reveal_batch_size=int(0.05 * n),
                                 refit_batch_size=int(0.05 * n),
                                 alpha_spending_fn='instantaneous',
                                 random_seed=random_seed)

strategy.run_strategy(verbose=True)

METRICS = {'curr_sample_efficiency': 1.0, 'spent_alpha': np.float64(0.0), 'remaining_alpha': np.float64(0.05), 'n_shrink_revealed': 116, 'n_left_in_region': 284, 'region_mass_estimate': np.float64(0.71)}
SUMMARY: rejected = True, spent_alpha = 0.050000000000000044, n_left_in_region = 284, region_mass-estimate = 0.71


In [9]:
protocol.testing_history

,stage_number,mean,variance,sample_size,test_stat,alloc_alpha,remaining_alpha,crit_val,rejection
0,-1,0.232159,5.948922,500,2.128392,0.00,0.05,inf,False
1,6,0.425972,5.868100,284,2.963406,0.05,0.00,1.644854,True


In [10]:
# Run data splitting
datasplit = DataSplittingStrategy(X=TX,
                                  Y=Y,
                                  train_ratio=0.2,
                                  learner=rf_learner,
                                  alpha=alpha,
                                  test_thresh=0,
                                  random_seed=random_seed)
datasplit.run_strategy()

In [11]:
# Data split rejection
datasplit.rejected

np.True_

In [12]:
# Chiseling region metrics
sampler.estimate_region_metrics(protocol.rejected_region)

(np.float64(0.1443905291526125),
 np.float64(0.69951),
 np.float64(0.2064166761770561),
 np.float64(0.009407306854545234),
 np.float64(0.0014498129531080898),
 np.float64(0.013441617026597047))

In [13]:
# Data region metrics
sampler.estimate_region_metrics(datasplit.region)

(np.float64(0.1397319904819037),
 np.float64(0.68119),
 np.float64(0.2051292451179608),
 np.float64(0.009945025675872857),
 np.float64(0.001473669514850599),
 np.float64(0.014592742572596547))

In [16]:
# settings = {"task_id": 0,
#             "dgp": "BenchTM",
#             "benchtm_path": benchtm_path,
#             "scenario": 1,
#             "n": n,
#             "outcome_transform": "aipw_intercept_fair",
#             "alpha": alpha,
#             "random_seed": random_seed,
#             "strategy": "ChiselingEqualAlpha",
#             "reveal_batch_size": max(1, int(0.05 * n)),
#             "refit_batch_size": max(1, int(0.05 * n)),
#             "test_thresh": 0,
#             "learner": "causal_random_forest_learner",
#             "n_sims": 10}

settings = {"task_id": 0,
            "dgp": "BenchTM",
            "benchtm_path": benchtm_path,
            "scenario": 1,
            "n": n,
            "outcome_transform": "aipw_intercept_fair",
            "alpha": alpha,
            "random_seed": random_seed,
            "strategy": "Chiseling",
            "alpha_spending_fn": "instantaneous",
            "boundary_strategy": "random",
            "n_burn_in": 0.05,
            "reveal_batch_size": max(1, int(0.05 * n)),
            "refit_batch_size": max(1, int(0.05 * n)),
            "test_thresh": 0,
            "learner": "causal_random_forest_learner",
            "n_sims": 10}

# settings = {"task_id": 0,
#             "dgp": "BenchTM",
#             "benchtm_path": benchtm_path,
#             "scenario": 1,
#             "n": n,
#             "outcome_transform": "aipw_intercept_fair",
#             "alpha": alpha,
#             "random_seed": random_seed,
#             "strategy": "TTestStrategy",
#             "reveal_batch_size": max(1, int(0.05 * n)),
#             "refit_batch_size": max(1, int(0.05 * n)),
#             "test_thresh": 0,
#             "learner": "causal_random_forest_learner",
#             "n_sims": 10}

In [17]:
benchmark = Benchmark(settings)
benchmark.simulate_batch()
benchmark.simulation_results_df.head()

,task_id,sim_id,rejected,region_mass,subgroup_mean,subgroup_utility,region_mass_se,subgroup_mean_se,subgroup_utility_se
0,0,0,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
1,0,1,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
2,0,2,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
3,0,3,True,0.64363,0.21619,0.139147,0.001514,0.016103,0.010369
4,0,4,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000


In [18]:
benchmark.simulation_results_df.rejected.mean()

np.float64(0.3)